In [ ]:
#@title Cell 1
from google.colab import drive
drive.mount('/content/drive')

# === THIẾT LẬP ĐƯỜNG DẪN (SỬA CHO KHỚP VỚI DRIVE CỦA BẠN) ===
PDF_PATH   = '/content/drive/MyDrive/YOLO-DocLayNet/Data/KTDK.pdf'
WORK_DIR   = '/content/drive/MyDrive/YOLO-DocLayNet/Outputs/KTDK'  # nơi lưu toàn bộ kết quả
IMAGES_DIR = f'{WORK_DIR}/images'    # crop ảnh/đồ thị
PAGES_DIR  = f'{WORK_DIR}/pdf_pages' # ảnh từng trang PDF
PRED_DIR   = f'{WORK_DIR}/pred_images' # ảnh có vẽ bbox (debug)
JSON_DIR   = f'{WORK_DIR}/pred_json'   # bbox theo từng trang
CHUNK_DIR  = f'{WORK_DIR}/chunks'      # text theo Chương

print("PDF_PATH =", PDF_PATH)
print("WORK_DIR =", WORK_DIR)


Mounted at /content/drive
PDF_PATH = /content/drive/MyDrive/YOLO-DocLayNet/Data/KTDK.pdf
WORK_DIR = /content/drive/MyDrive/YOLO-DocLayNet/Outputs/KTDK


In [ ]:
#@title Cell2
%cd /content
!git clone https://github.com/ppaanngggg/yolo-doclaynet.git
%cd /content/yolo-doclaynet

!pip install -U pip
!pip install -r requirements.txt

# Xử lý PDF, ảnh, hiển thị
!pip install pypdfium2 pymupdf opencv-python matplotlib tqdm


/content
Cloning into 'yolo-doclaynet'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 164 (delta 50), reused 46 (delta 32), pack-reused 95 (from 1)
Receiving objects: 100% (164/164), 51.83 KiB | 804.00 KiB/s, done.
Resolving deltas: 100% (83/83), done.
/content/yolo-doclaynet
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [ultralytics]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 15.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 74.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#@title Cell3
import os, json, re, shutil
from pathlib import Path

# Nhãn DocLayNet
DOCLAYNET_CLASSES = [
    "Text", "Picture", "Caption", "Section-header", "Footnote",
    "Formula", "Table", "List-item", "Page-header", "Page-footer", "Title"
]
# Chúng ta coi "Picture" (và tùy chọn "Table") là ảnh cần cắt
IMAGE_LIKE_CLASSES = {"Picture", "Table"}  # nếu không muốn cắt Table, bỏ "Table" đi

# Tạo thư mục
for d in [WORK_DIR, IMAGES_DIR, PAGES_DIR, PRED_DIR, JSON_DIR, CHUNK_DIR]:
    os.makedirs(d, exist_ok=True)

def save_predictions_to_json(preds, out_json_path):
    """Lưu bbox theo định dạng gọn: [{cls_name, conf, bbox_xyxy, page_image}]"""
    items = []
    for r in preds:
        boxes = r.boxes
        for b in boxes:
            xyxy   = b.xyxy[0].tolist()
            conf   = float(b.conf[0].item())
            cls_id = int(b.cls[0].item())
            cls_nm = DOCLAYNET_CLASSES[cls_id] if 0 <= cls_id < len(DOCLAYNET_CLASSES) else str(cls_id)
            items.append({
                "cls_name": cls_nm,
                "conf": conf,
                "bbox_xyxy": [float(x) for x in xyxy],
            })
    with open(out_json_path, "w", encoding="utf-8") as f:
        json.dump(items, f, ensure_ascii=False, indent=2)


In [ ]:
#@title Cell4
import pypdfium2 as pdfium
from tqdm import tqdm

def pdf_to_images(pdf_path, out_dir, scale=2.0):
    pdf = pdfium.PdfDocument(pdf_path)
    paths = []
    for i in tqdm(range(len(pdf)), desc="Rendering PDF"):
        page = pdf.get_page(i)
        pil  = page.render(scale=scale).to_pil()
        pth  = os.path.join(out_dir, f"page_{i+1:04d}.png")
        pil.save(pth)
        paths.append(pth)
        page.close()
    return paths

page_images = pdf_to_images(PDF_PATH, PAGES_DIR, scale=2.0)
print("Tổng số trang:", len(page_images))
print("Ví dụ:", page_images[:3])


Rendering PDF: 100%|██████████| 207/207 [00:35<00:00,  5.86it/s]

Tổng số trang: 207
Ví dụ: ['/content/drive/MyDrive/YOLO-DocLayNet/Outputs/KTDK/pdf_pages/page_0001.png', '/content/drive/MyDrive/YOLO-DocLayNet/Outputs/KTDK/pdf_pages/page_0002.png', '/content/drive/MyDrive/YOLO-DocLayNet/Outputs/KTDK/pdf_pages/page_0003.png']


In [ ]:
#@title Cell5
from ultralytics import YOLO

# Dùng trọng số có sẵn trong repo (YOLOv9t). Bạn có thể thay bằng bản khác nếu có.
MODEL_PATH = '/content/yolo-doclaynet/yolo-doclaynet.pt'
model = YOLO(MODEL_PATH)
model


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): ELAN1(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=Tru

In [ ]:
#@title Cell6 (dedup) — ưu tiên Formula khi trùng bbox, đọc từ predict/labels
import os, shutil, json, cv2
from pathlib import Path
from tqdm import tqdm

TARGET_CLASSES = {"Formula", "Picture", "Table"}
CONF_THRES = 0.25
IOU_THRES  = 0.50

# ---- Helper
def iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    ix1, iy1 = max(ax1, bx1), max(ay1, by1)
    ix2, iy2 = min(ax2, bx2), min(ay2, by2)
    iw, ih = max(0, ix2 - ix1), max(0, iy2 - iy1)
    inter = iw * ih
    if inter <= 0: return 0.0
    area_a = max(0, ax2 - ax1) * max(0, ay2 - ay1)
    area_b = max(0, bx2 - bx1) * max(0, by2 - by1)
    return inter / (area_a + area_b - inter + 1e-6)

def dedup_keep_formula(dets, iou_thr=0.90):
    """
    dets: list of dict {cls_name, conf, bbox_xyxy}
    - Nếu hai bbox trùng (IoU >= iou_thr) và có Formula vs Table/Picture -> chỉ giữ Formula.
    - Nếu trùng cùng lớp -> giữ conf cao hơn.
    Trả về danh sách đã lọc.
    """
    kept = []
    used = [False]*len(dets)
    for i in range(len(dets)):
        if used[i]: continue
        a = dets[i]
        group = [i]
        # gom các box trùng mạnh với a
        for j in range(i+1, len(dets)):
            if used[j]: continue
            b = dets[j]
            if iou_xyxy(a["bbox_xyxy"], b["bbox_xyxy"]) >= iou_thr:
                group.append(j)
        # quyết định giữ gì trong group
        cand = [dets[k] for k in group]
        # nếu có Formula -> chọn Formula có conf cao nhất
        form = [c for c in cand if c["cls_name"] == "Formula"]
        if form:
            best = max(form, key=lambda x: x["conf"])
        else:
            # không có Formula: chọn conf cao nhất trong group
            best = max(cand, key=lambda x: x["conf"])
        kept.append(best)
        for k in group: used[k] = True
    return kept

# ---- 1) đảm bảo dự đoán hiện tại ở đúng thư mục predict
pred_run_dir = Path(PRED_DIR) / "predict"
labels_dir   = pred_run_dir / "labels"
if not pred_run_dir.exists() or not labels_dir.exists():
    # xoá nếu cũ để tránh predict2, rồi chạy lại predict
    if pred_run_dir.exists(): shutil.rmtree(pred_run_dir)
    _ = model.predict(
        page_images,
        conf=CONF_THRES,
        iou=IOU_THRES,
        save=True,
        save_txt=True,
        save_conf=True,
        project=PRED_DIR,
        name="predict",
        exist_ok=False,
        verbose=False
    )
    labels_dir = pred_run_dir / "labels"
assert labels_dir.exists(), f"Không thấy thư mục nhãn: {labels_dir}"

# ---- 2) chuẩn bị mapping tên lớp từ mô hình (chữa lệch tên)
id2name = model.names if isinstance(model.names, dict) else {i:n for i,n in enumerate(model.names)}

# ---- 3) crop theo labels (sau khi dedup)
os.makedirs(IMAGES_DIR, exist_ok=True)
os.makedirs(JSON_DIR, exist_ok=True)
manifest = []

def _new_counter():
    return {"Picture":0, "Table":0, "Formula":0}

for img_path in tqdm(page_images, desc="Cropping with formula-priority"):
    img_path = Path(img_path)
    stem = img_path.stem
    lab_path = labels_dir / f"{stem}.txt"

    img = cv2.imread(str(img_path))
    if img is None:
        print(f"[WARN] Không đọc được ảnh: {img_path}")
        # vẫn ghi JSON rỗng
        with open(Path(JSON_DIR) / f"{stem}.json", "w", encoding="utf-8") as f:
            json.dump([], f, ensure_ascii=False, indent=2)
        continue
    h, w = img.shape[:2]

    raw = []
    if lab_path.exists():
        with open(lab_path, "r", encoding="utf-8") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 5:
                    continue
                cls_id = int(float(parts[0]))
                cls_nm = id2name.get(cls_id, str(cls_id))
                if cls_nm not in TARGET_CLASSES:
                    continue
                cx = float(parts[1]) * w
                cy = float(parts[2]) * h
                bw = float(parts[3]) * w
                bh = float(parts[4]) * h
                conf = float(parts[5]) if len(parts) >= 6 else 1.0

                x1 = int(round(cx - bw/2)); y1 = int(round(cy - bh/2))
                x2 = int(round(cx + bw/2)); y2 = int(round(cy + bh/2))
                x1 = max(0, min(x1, w-1)); x2 = max(0, min(x2, w-1))
                y1 = max(0, min(y1, h-1)); y2 = max(0, min(y2, h-1))
                if x2 <= x1 or y2 <= y1:
                    continue

                raw.append({"cls_name": cls_nm, "conf": conf, "bbox_xyxy": [x1,y1,x2,y2]})

    # ưu tiên Formula khi trùng
    dets = dedup_keep_formula(raw, iou_thr=0.90)

    # lưu JSON theo trang (đã lọc)
    with open(Path(JSON_DIR) / f"{stem}.json", "w", encoding="utf-8") as f:
        json.dump(dets, f, ensure_ascii=False, indent=2)

    # crop & ghi manifest
    counters = _new_counter()
    for d in dets:
        x1, y1, x2, y2 = d["bbox_xyxy"]
        crop = img[y1:y2, x1:x2]
        if crop.size == 0:
            continue
        counters[d["cls_name"]] += 1
        kind = d["cls_name"].lower().replace("-", "_")
        out_crop = Path(IMAGES_DIR) / f"{stem}_{kind}_{counters[d['cls_name']]:02d}.png"
        cv2.imwrite(str(out_crop), crop)

        manifest.append({
            "page": stem,
            "crop_path": str(out_crop),
            "cls_name": d["cls_name"],
            "conf": float(d["conf"]),
            "bbox_xyxy": [int(x1), int(y1), int(x2), int(y2)],
            "source": "labels+dedup"
        })

# ---- 4) lưu MANIFEST tổng
MANIFEST_PATH = os.path.join(WORK_DIR, "MANIFEST.json")
with open(MANIFEST_PATH, "w", encoding="utf-8") as f:
    json.dump(manifest, f, ensure_ascii=False, indent=2)

print("Done. JSON & crops đã ưu tiên Formula khi trùng.")
print("Labels dir:", labels_dir)
print("Crops:", IMAGES_DIR)
print("Per-page JSON:", JSON_DIR)
print("Manifest:", MANIFEST_PATH)


Results saved to /content/drive/MyDrive/YOLO-DocLayNet/Outputs/KTDK/pred_images/predict
207 labels saved to /content/drive/MyDrive/YOLO-DocLayNet/Outputs/KTDK/pred_images/predict/labels


Cropping with formula-priority: 100%|██████████| 207/207 [00:15<00:00, 13.76it/s]

Done. JSON & crops đã ưu tiên Formula khi trùng.
Labels dir: /content/drive/MyDrive/YOLO-DocLayNet/Outputs/KTDK/pred_images/predict/labels
Crops: /content/drive/MyDrive/YOLO-DocLayNet/Outputs/KTDK/images
Per-page JSON: /content/drive/MyDrive/YOLO-DocLayNet/Outputs/KTDK/pred_json
Manifest: /content/drive/MyDrive/YOLO-DocLayNet/Outputs/KTDK/MANIFEST.json


In [ ]:
#@title Cell7 — Gom theo trục Y, <image> cho Formula/Picture/Table; TÁCH CHƯƠNG & PHỤ LỤC; lưu từ chapter_0.txt
import os, re, json, cv2, fitz
from pathlib import Path
from typing import List, Dict, Any, Tuple

IMAGE_TOKEN = "<image>"

# ==== Đường dẫn & lớp ảnh ====
pred_run_dir = Path(PRED_DIR) / "predict"
labels_dir   = pred_run_dir / "labels"
assert labels_dir.exists(), f"Không tìm thấy labels dir: {labels_dir}"

IMAGE_LIKE = {"Formula", "Picture", "Table"}

# ==== Dùng mapping lớp từ chính mô hình (đã load ở Cell5) ====
id2name = model.names if isinstance(model.names, dict) else {i:n for i,n in enumerate(model.names)}

# ==== Regex tách heading ====
RE_PAGE_MARK     = re.compile(r"^\[--- PAGE\s+(\d+)\s+---\]$")

# Chương
RE_HEAD_CH_ONLY   = re.compile(r"^\s*(?:CHƯƠNG|Chương|CHAPTER|Chapter)\s+([IVXLCDM]+|\d+)\s*$")
RE_HEAD_CH_INLINE = re.compile(r"^\s*(?:CHƯƠNG|Chương|CHAPTER|Chapter)\s+([IVXLCDM]+|\d+)\s*[\.\-–:]?\s+(.+?)\s*$")

# Phụ lục
RE_HEAD_APP_ONLY   = re.compile(r"^\s*(?:PHỤ LỤC|Phụ lục|APPENDIX|Appendix)\s+([A-Z]+)\s*$")
RE_HEAD_APP_INLINE = re.compile(r"^\s*(?:PHỤ LỤC|Phụ lục|APPENDIX|Appendix)\s+([A-Z]+)\s*[\.\-–:]?\s+(.+?)\s*$")

# ==== Tiện ích gom block ====
def page_key(i0: int) -> str:
    return f"page_{i0+1:04d}"

def read_all_labels_for_page(stem: str, iw: int, ih: int) -> List[Dict[str, Any]]:
    path = labels_dir / f"{stem}.txt"
    dets = []
    if not path.exists():
        return dets
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            ps = line.strip().split()
            if len(ps) < 5:
                continue
            cls_id = int(float(ps[0]))
            cls_nm = id2name.get(cls_id, str(cls_id))
            cx = float(ps[1]) * iw
            cy = float(ps[2]) * ih
            bw = float(ps[3]) * iw
            bh = float(ps[4]) * ih
            conf = float(ps[5]) if len(ps) >= 6 else 1.0
            x1 = int(round(cx - bw/2)); y1 = int(round(cy - bh/2))
            x2 = int(round(cx + bw/2)); y2 = int(round(cy + bh/2))
            x1 = max(0, min(x1, iw-1)); x2 = max(0, min(x2, iw-1))
            y1 = max(0, min(y1, ih-1)); y2 = max(0, min(y2, ih-1))
            if x2 <= x1 or y2 <= y1:
                continue
            dets.append({
                "cls_name": cls_nm,
                "conf": conf,
                "bbox_xyxy": [x1,y1,x2,y2],
                "h": (y2 - y1),
                "y0": y1
            })
    return dets

def sort_blocks_y_then_height(blocks: List[Dict[str,Any]]) -> List[Dict[str,Any]]:
    EPS_Y = 5
    blocks = sorted(blocks, key=lambda d: (d["y0"], -d["h"]))
    out = []
    i = 0
    n = len(blocks)
    while i < n:
        j = i + 1
        group = [blocks[i]]
        while j < n and abs(blocks[j]["y0"] - blocks[i]["y0"]) <= EPS_Y:
            group.append(blocks[j]); j += 1
        group.sort(key=lambda d: (-d["h"], d["bbox_xyxy"][0]))  # cao hơn trước; cùng cao thì trái→phải
        out.extend(group)
        i = j
    return out

def clip_text_in_bbox(doc_page, bbox_xyxy, iw, ih) -> str:
    x1,y1,x2,y2 = bbox_xyxy
    W_pdf, H_pdf = doc_page.rect.width, doc_page.rect.height
    sx, sy = W_pdf/iw, H_pdf/ih
    rect = fitz.Rect(x1*sx, y1*sy, x2*sx, y2*sy)
    txt = doc_page.get_text("text", clip=rect) or ""
    lines = [ln.strip() for ln in txt.splitlines()]
    lines = [ln for ln in lines if ln]
    return "\n".join(lines).strip()

# ==== Duyệt từng trang, build tuyến tính theo trục Y ====
doc = fitz.open(PDF_PATH)
linear_lines: List[str] = []

for i in range(len(doc)):
    pn = i + 1
    stem = page_key(i)
    linear_lines.append(f"[--- PAGE {pn} ---]")

    img_path = Path(PAGES_DIR) / f"{stem}.png"
    im = cv2.imread(str(img_path))
    if im is None:
        full_txt = doc[i].get_text("text") or ""
        linear_lines.extend([ln for ln in full_txt.splitlines()])
        linear_lines.append("")
        continue
    ih, iw = im.shape[:2]

    blocks = read_all_labels_for_page(stem, iw, ih)
    if not blocks:
        full_txt = doc[i].get_text("text") or ""
        linear_lines.extend([ln for ln in full_txt.splitlines()])
        linear_lines.append("")
        continue

    blocks_sorted = sort_blocks_y_then_height(blocks)

    page_lines = []
    p = doc[i]
    for b in blocks_sorted:
        if b["cls_name"] in IMAGE_LIKE:
            page_lines.append(IMAGE_TOKEN)
        else:
            txt = clip_text_in_bbox(p, b["bbox_xyxy"], iw, ih)
            if txt:
                page_lines.append(txt)

    linear_lines.extend(page_lines)
    linear_lines.append("")

# =======================
#   TÁCH CHƯƠNG & PHỤ LỤC
# =======================
def next_non_empty(lines: List[str], i: int, within: int = 8):
    j = i + 1; steps = 0
    while j < len(lines) and steps < within:
        if (lines[j] or "").strip():
            return j
        j += 1; steps += 1
    return None

chapters: List[Dict[str, Any]] = []
current = {"title": "Chương 0. Mở đầu", "lines": []}

def flush():
    chapters.append(current.copy())
    current.clear()
    current.update({"title": None, "lines": []})

i, n = 0, len(linear_lines)
while i < n:
    raw = linear_lines[i]
    ln  = (raw or "").strip()

    m_page = RE_PAGE_MARK.match(ln)
    if m_page:
        # ==== 1) Thử PHỤ LỤC trước ====
        j1 = next_non_empty(linear_lines, i, within=8)
        if j1 is not None:
            cand = linear_lines[j1] or ""

            m_app_inline = RE_HEAD_APP_INLINE.match(cand)
            if m_app_inline:
                flush()
                app = m_app_inline.group(1)
                title = m_app_inline.group(2).strip()
                current = {"title": f"Phụ lục {app}. {title}" if title else f"Phụ lục {app}", "lines": []}
                i += 1
                continue

            m_app_only = RE_HEAD_APP_ONLY.match(cand)
            if m_app_only:
                j2 = next_non_empty(linear_lines, j1, within=6)
                if j2 is not None:
                    title_line = (linear_lines[j2] or "").strip()
                    # tránh trùng với marker/heading khác
                    if (not RE_PAGE_MARK.match(title_line)) and (not RE_HEAD_APP_ONLY.match(title_line)) and (not RE_HEAD_CH_ONLY.match(title_line)):
                        flush()
                        app = m_app_only.group(1)
                        current = {"title": f"Phụ lục {app}. {title_line}", "lines": []}
                        i += 1
                        continue

        # ==== 2) Nếu không phải Phụ lục, thử CHƯƠNG ====
        if j1 is not None:
            cand = linear_lines[j1] or ""

            m_ch_inline = RE_HEAD_CH_INLINE.match(cand)
            if m_ch_inline:
                flush()
                num   = m_ch_inline.group(1)
                title = m_ch_inline.group(2).strip()
                current = {"title": f"Chương {num}. {title}" if title else f"Chương {num}", "lines": []}
                i += 1
                continue

            m_ch_only = RE_HEAD_CH_ONLY.match(cand)
            if m_ch_only:
                j2 = next_non_empty(linear_lines, j1, within=6)
                if j2 is not None:
                    title_line = (linear_lines[j2] or "").strip()
                    if (not RE_PAGE_MARK.match(title_line)) and (not RE_HEAD_CH_ONLY.match(title_line)) and (not RE_HEAD_APP_ONLY.match(title_line)):
                        flush()
                        num = m_ch_only.group(1)
                        current = {"title": f"Chương {num}. {title_line}", "lines": []}
                        i += 1
                        continue

    # Không phải mở đầu chương/phụ lục → ghi vào chương hiện tại
    current["lines"].append(raw)
    i += 1

# đóng chương cuối
chapters.append(current)

if len(chapters) == 1 and chapters[0]["title"] == "Chương 0. Mở đầu":
    chapters[0]["title"] = "Chương 0. Toàn văn"

# ==== Ghi ra files: bắt đầu từ chapter_0.txt ====
os.makedirs(CHUNK_DIR, exist_ok=True)
for idx, ch in enumerate(chapters):  # bắt đầu từ 0
    out_txt = os.path.join(CHUNK_DIR, f"chapter_{idx}.txt")
    with open(out_txt, "w", encoding="utf-8") as f:
        f.write(ch["title"].strip() + "\n\n")
        f.write("\n".join(ch["lines"]).strip() + "\n")

CHAPTER_META = os.path.join(WORK_DIR, "CHAPTERS.json")
with open(CHAPTER_META, "w", encoding="utf-8") as f:
    json.dump(chapters, f, ensure_ascii=False, indent=2)

print(f"Tách được {len(chapters)} phần (bao gồm Chương/Phụ lục). Lưu từ chapter_0.txt trong:", CHUNK_DIR)
print("CHAPTERS.json:", CHAPTER_META)

Tách được 14 phần (bao gồm Chương/Phụ lục). Lưu từ chapter_0.txt trong: /content/drive/MyDrive/YOLO-DocLayNet/Outputs/KTDK/chunks
CHAPTERS.json: /content/drive/MyDrive/YOLO-DocLayNet/Outputs/KTDK/CHAPTERS.json
